
<h1 id="Practical-Data-Science-2016,-Assignment-1">Practical Data Science 2016, Assignment 2<a class="anchor-link" href="#Practical-Data-Science-2016,-Assignment-1">&#182;</a></h1>
<p>In this assignment, we  follow the steps of researchers that investigated <a href="http://dx.doi.org/10.1016/j.jfineco.2015.10.001">The value of connections in turbulent times: Evidence from the United States</a>.

<blockquote><p>Dimitrios Karamanis <br />
Department of Informatics <br />
Athens University of Economics and Business <br />
dkaramanis@aueb.gr</p>
</blockquote>

<h2 id="Question-1">Reading the data<a class="anchor-link" href="#Question-1">&#182;</a></h2>
<p>From  <a href="http://economics.mit.edu/faculty/acemoglu/data/connections">Acemoglu's site</a>, we download and read the paper's replication files, and more specially financial and return stata/dta data:</p>

In [1]:
#import libraries
import pandas as pd
import numpy as np
from scipy import stats
import pandas_datareader.data as pdr;
from datetime import datetime
import statsmodels.formula.api as smf

#read the data
companies=pd.read_stata('C:/Users/Owner/Desktop/5. Practical Data Science/Assignment 2/Data and Programs/Stata Files/Financial Data.dta')
returns=pd.read_stata('C:/Users/Owner/Desktop/5. Practical Data Science/Assignment 2/Data and Programs/Stata Files/Return Data.dta')

companies.head()

,ticker,name,ta2008,roe2008,tdtc2008,mcap2008,tdep2ta2008,tdep2ta2007,addresscity_ds,addressstate_ds,siccode1,fullsample
0,AACC,ASSET ACCEP.CAP.CORP.,408171.0,12.14,57.060001,156228.0,0.000000,0.000000,Warren,MICHIGAN,6153.0,1.0
1,AANB,ABIGAIL ADAMS NAT.BANC.,417655.0,-20.74,67.580002,8694.0,NaN,87.260002,Washington,DISTRICT OF COLUMBIA,6021.0,1.0
2,AB,ALLBERN.HLDG.LP.,1601442.0,15.49,0.000000,1875752.0,0.000000,0.000000,New York,NEW YORK,6282.0,1.0
3,ABBC,ABINGTON BANCORP INCO.,1185297.0,0.87,53.570000,216172.0,56.099998,56.560001,Jenkintown,PENNSYLVANIA,6036.0,1.0
4,ABCB,AMERIS BANCORP,2399592.0,-2.22,37.180000,160385.0,83.910004,0.000000,SE Moultrie,GEORGIA,6021.0,1.0


In [2]:
returns.head()

,index,ticker,ri31dec04,ri3jan05,ri4jan05,ri5jan05,ri6jan05,ri7jan05,ri10jan05,ri11jan05,...,ri16feb09,ri17feb09,ri18feb09,ri19feb09,ri20feb09,ri23feb09,ri24feb09,ri25feb09,ri26feb09,ri27feb09
0,1,DS1,147.199997,147.199997,147.199997,147.199997,147.199997,147.199997,139.250000,138.320007,...,72.900002,72.900002,60.750000,66.360001,66.360001,67.099998,67.099998,67.290001,67.290001,67.290001
1,2,FCCY,472.220001,468.160004,468.160004,465.839996,464.440002,470.679993,459.459991,454.609985,...,211.289993,202.529999,186.919998,190.910004,194.559998,194.559998,186.220001,184.270004,190.910004,184.600006
2,3,FPBN,203.320007,203.320007,190.869995,190.869995,190.460007,190.869995,190.869995,178.419998,...,29.879999,29.879999,29.879999,29.209999,29.209999,20.250000,24.480000,20.910000,20.750000,16.600000
3,4,SRCE,6233.529785,6189.540039,5932.970215,5840.109863,5818.120117,5676.390137,5666.620117,5671.509766,...,5769.700195,5383.669922,5241.140137,5217.379883,5155.020020,4825.410156,5342.100098,5223.319824,5371.790039,5309.430176
4,5,FCF,327.850006,325.500000,318.690002,314.209991,315.059998,308.890015,310.589996,310.799988,...,252.130005,235.690002,227.210007,219.520004,223.759995,218.460007,232.250000,231.720001,226.940002,216.869995


<p>In their paper Daron Acemoglu, Simon Johnson, Amir Kermani, James Kwak, and Todd Mitton investigated whether firms that had personal connections with Timothy Geithner experienced an abnormal return when Geithner was appointed as U.S. Treasury Secretary on November 21, 2008.</p>
<p>The investigation will focus on the days following event day, where the event was Geithner's appointment: event day 0 was the day of the appointment, event day 1 was the following day, and so on.</p>
<p>Let's replicate part from their work for a sample of Firms.</p>

In [3]:
Firm=[
'BANK OF NY.MELLON CORP.',
'BEACON FED.BANC.INCORP.',
'BLACKROCK INCO.',
'CARVER BANCORP INCO.',
'CITIGROUP INCO.',
'CME GROUP INCO.',
'EVERCORE PARTNERS INCO.',
'FEDERATED INVRS.INCO.',
'FORTRESS INV.GP.LLC.',
'GAMCO INVESTORS INCO.',
'LAZARD LTD.',
'NORTHERN TRUST CORP.',
'POPULAR INCO.',
'PROVIDENT FINL.SVS.INCO.',
'STATE STREET CORP.',
'THE BLACKSTONE GROUP LP.',
'THE NASDAQ OMX GP.INCO.']

companies=companies[companies['name'].isin(Firm)]
companies

,ticker,name,ta2008,roe2008,tdtc2008,mcap2008,tdep2ta2008,tdep2ta2007,addresscity_ds,addressstate_ds,siccode1,fullsample
57,BFED,BEACON FED.BANC.INCORP.,1.011947e+06,-2.790000,73.870003,60651.0,61.910000,58.660000,East Syracuse,NEW YORK,6035.0,1.0
63,BK,BANK OF NY.MELLON CORP.,2.375120e+08,5.090000,45.820000,32536080.0,67.300003,59.790001,New York,NEW YORK,6022.0,1.0
70,BLK,BLACKROCK INCO.,1.991301e+07,6.640000,8.360000,15734600.0,0.000000,0.000000,New York,NEW YORK,6282.0,1.0
83,BPOP,POPULAR INCO.,3.852526e+07,-49.759998,67.989998,1455144.0,71.510002,64.559998,San Juan,#NA,6021.0,1.0
89,BX,THE BLACKSTONE GROUP LP.,8.412488e+06,-30.070000,6.160000,1741566.0,0.000000,0.000000,New York,NEW YORK,6282.0,1.0
92,C,CITIGROUP INCO.,1.938470e+09,-28.469999,82.540001,36569950.0,39.939999,37.770000,New York,NEW YORK,6021.0,1.0
100,CARV,CARVER BANCORP INCO.,7.883080e+05,7.510000,44.410000,12409.0,83.050003,83.930000,New York,NEW YORK,6035.0,1.0
140,CME,CME GROUP INCO.,4.815866e+07,4.620000,16.420000,13822670.0,0.000000,0.000000,Chicago,ILLINOIS,6231.0,1.0
232,EVR,EVERCORE PARTNERS INCO.,6.778740e+05,-2.490000,29.150000,150546.0,0.000000,0.000000,New York,NEW YORK,6282.0,1.0
275,FIG,FORTRESS INV.GP.LLC.,1.169669e+06,-236.380005,82.589996,406681.0,0.000000,0.000000,New York,NEW YORK,6726.0,1.0


In [4]:
#take the abbreviations of the Firms
abbr=companies['ticker']
abbr

57     BFED
63       BK
70      BLK
83     BPOP
89       BX
92        C
100    CARV
140     CME
232     EVR
275     FIG
276     FII
313     GBL
387     LAZ
438    NDAQ
454    NTRS
500     PFS
587     STT
Name: ticker, dtype: object

<h2 id="Question-1">Question 1<a class="anchor-link" href="#Question-1">&#182;</a></h2><i><p>Investigate the returns of the selected companies compared to the returns of the S&amp;P 500 index for event days 0, 1, ..., 10, and the cumulative returns for all event days 0 to 10. So, you answer must include a table with a row for each of the event days and the cumulative result. It must also include columns for the set of the selected companies, the S&amp;P 500, and the result of the statistical set that you used to determine the statistical significance between the two figures for each row.  Your anwer will be similar in spirit to Table 2 of the original paper, however you will be using the whole S&amp;P instead of the non-connected companies that is used there.</p></i>

<p>In this part we will use two different sources to access data. The first is via yahooo_finance:</p>

In [5]:
# get prices for SP500 for the days we want
from yahoo_finance import Share
sp500 = Share('^GSPC') #abbreviation of S&P500 
sp500=sp500.get_historical('2014-11-21','2014-12-08')
sp500=pd.DataFrame(sp500)
sp500=sp500[['Close','Open']]
sp500= sp500.astype(float)
sp500['sp500']=(sp500['Close']-sp500['Open'])/sp500['Open'] #calculate return of SP500
del sp500['Close']
del sp500['Open']
df=sp500.copy() #create my main dataframe
df

,sp500
0,-0.007003
1,0.001250
2,-0.000829
3,0.003328
4,0.006223
5,-0.005974
6,-0.003480
7,0.002646
8,-0.001507
9,0.002102


<p>We define a function that brings back a stock's return for the days of interest</p>

In [6]:
#define Function that brings back the Share's return
def share_return(x):
    a=Share(x)
    a=a.get_historical('2014-11-21','2014-12-08')
    a=pd.DataFrame(a)
    a=a[['Close','Open']]
    a= a.astype(float)
    a['return']=(a['Close']-a['Open'])/a['Open'] #stock's return
    del a['Close']
    del a['Open']
    a.rename(columns={'return': x},inplace=True)
    return a;

In [7]:
#BFED is not recognised in yahoo_finance so we create new list only for the rest of the Stocks
list=abbr[1:17] 
for i in list:    
    x=share_return(i)
    df=pd.concat([df,x], axis=1)    
df

,sp500,BK,BLK,BPOP,BX,C,CARV,CME,EVR,FIG,FII,GBL,LAZ,NDAQ,NTRS,PFS,STT
0,-0.007003,-0.000727,0.005141,0.004530,-0.000589,0.006967,0.006623,0.011028,0.023485,0.000000,0.004233,-0.004569,0.025546,0.003688,0.004829,-0.001145,0.006058
1,0.001250,0.013061,0.004382,0.015291,0.009218,0.010086,0.000000,0.012436,0.016359,-0.002525,0.025379,-0.000794,0.001367,-0.006957,0.005442,0.011521,0.005198
2,-0.000829,0.002981,0.014912,-0.007634,0.010856,0.003820,0.000000,0.009372,0.014835,-0.003783,0.012602,0.019399,0.003737,0.006573,-0.001035,0.000577,0.001535
3,0.003328,0.004725,0.004249,0.015152,-0.000904,0.013585,0.000000,0.015810,0.045612,0.005083,0.014336,0.001838,0.009919,0.010204,0.008923,0.003462,0.013059
4,0.006223,0.013078,-0.000730,0.003100,-0.002417,0.015885,0.000000,0.011341,0.000405,0.008974,0.010943,0.023514,0.009780,0.013264,0.009699,0.004643,0.012291
5,-0.005974,-0.009262,-0.004738,-0.010154,-0.013514,-0.008364,0.000000,0.000236,-0.017764,0.010363,-0.012472,-0.008212,-0.016490,-0.012932,-0.010520,-0.003482,0.002902
6,-0.003480,-0.004229,-0.000501,-0.008204,0.000597,-0.000556,0.000000,0.006182,-0.010774,-0.005148,-0.011010,-0.013284,0.023034,0.002008,-0.000443,-0.024280,-0.003506
7,0.002646,0.003995,-0.001536,0.002749,0.003896,-0.005877,0.000000,0.001077,-0.019661,0.011658,-0.000315,0.010403,-0.000593,0.005834,-0.005004,-0.005070,-0.002983
8,-0.001507,-0.006196,0.001037,0.002751,0.002402,-0.001287,0.027397,0.006264,0.009743,-0.015287,0.001263,0.017730,-0.005697,-0.001120,-0.002206,-0.001130,-0.002068
9,0.002102,-0.002988,-0.004310,0.013383,0.000000,0.008528,0.000000,-0.002393,0.001377,0.001276,0.009920,0.002451,-0.007409,0.008130,-0.003237,0.013211,0.007832


<p>We create a column with the dates of interest</p>

In [8]:
date=['2008-11-21',
      '2008-11-24',
      '2008-11-25',
      '2008-11-26',
      '2008-11-28',
      '2008-12-01',
      '2008-12-02',
      '2008-12-03',
      '2008-12-04',
      '2008-12-05',
      '2008-12-08'
      ]
df['Date']=date

<p>We download data for BFED from pandas_datareader: </p>

In [9]:
# we download data for BFED from pandas_datareader and calculate return
bfed = pdr.get_data_yahoo(symbols='BFED', start=datetime(2008, 11, 21),end=datetime(2008, 12, 8))
bfed=pd.DataFrame(bfed)
bfed=bfed[['Close','Open']]
bfed['BFED']=(bfed['Close']-bfed['Open'])/bfed['Open']
del bfed['Close']
del bfed['Open']
bfed

,BFED
Date,
2008-11-21,-0.001238
2008-11-24,0.000013
2008-11-26,-0.001214
2008-11-28,0.054545
2008-12-01,-0.001189
2008-12-02,-0.005441
2008-12-03,0.000000
2008-12-04,0.083871
2008-12-05,0.000000


<p>as we see BFED it has no values for  25/11/08 and 08/12/08. We will merge the new variable with our dataframe and fill the missing values: </p>


In [10]:
# as we see it has no values for  25/11/08 and 08/12/08
#we create a new colum with dates existed
bfed_dates=['2008-11-21',
            '2008-11-24',
            '2008-11-26',
            '2008-11-28',
            '2008-12-01',
            '2008-12-02',
            '2008-12-03',
            '2008-12-04',
            '2008-12-05'
            ]

bfed['Date']=bfed_dates
df=pd.merge(df,bfed,how='left', on=['Date'])
df

,sp500,BK,BLK,BPOP,BX,C,CARV,CME,EVR,FIG,FII,GBL,LAZ,NDAQ,NTRS,PFS,STT,Date,BFED
0,-0.007003,-0.000727,0.005141,0.004530,-0.000589,0.006967,0.006623,0.011028,0.023485,0.000000,0.004233,-0.004569,0.025546,0.003688,0.004829,-0.001145,0.006058,2008-11-21,-0.001238
1,0.001250,0.013061,0.004382,0.015291,0.009218,0.010086,0.000000,0.012436,0.016359,-0.002525,0.025379,-0.000794,0.001367,-0.006957,0.005442,0.011521,0.005198,2008-11-24,0.000013
2,-0.000829,0.002981,0.014912,-0.007634,0.010856,0.003820,0.000000,0.009372,0.014835,-0.003783,0.012602,0.019399,0.003737,0.006573,-0.001035,0.000577,0.001535,2008-11-25,NaN
3,0.003328,0.004725,0.004249,0.015152,-0.000904,0.013585,0.000000,0.015810,0.045612,0.005083,0.014336,0.001838,0.009919,0.010204,0.008923,0.003462,0.013059,2008-11-26,-0.001214
4,0.006223,0.013078,-0.000730,0.003100,-0.002417,0.015885,0.000000,0.011341,0.000405,0.008974,0.010943,0.023514,0.009780,0.013264,0.009699,0.004643,0.012291,2008-11-28,0.054545
5,-0.005974,-0.009262,-0.004738,-0.010154,-0.013514,-0.008364,0.000000,0.000236,-0.017764,0.010363,-0.012472,-0.008212,-0.016490,-0.012932,-0.010520,-0.003482,0.002902,2008-12-01,-0.001189
6,-0.003480,-0.004229,-0.000501,-0.008204,0.000597,-0.000556,0.000000,0.006182,-0.010774,-0.005148,-0.011010,-0.013284,0.023034,0.002008,-0.000443,-0.024280,-0.003506,2008-12-02,-0.005441
7,0.002646,0.003995,-0.001536,0.002749,0.003896,-0.005877,0.000000,0.001077,-0.019661,0.011658,-0.000315,0.010403,-0.000593,0.005834,-0.005004,-0.005070,-0.002983,2008-12-03,0.000000
8,-0.001507,-0.006196,0.001037,0.002751,0.002402,-0.001287,0.027397,0.006264,0.009743,-0.015287,0.001263,0.017730,-0.005697,-0.001120,-0.002206,-0.001130,-0.002068,2008-12-04,0.083871
9,0.002102,-0.002988,-0.004310,0.013383,0.000000,0.008528,0.000000,-0.002393,0.001377,0.001276,0.009920,0.002451,-0.007409,0.008130,-0.003237,0.013211,0.007832,2008-12-05,0.000000


In [11]:
#as we see BFED has missing values, we will fill it with the previous value
df['BFED']=df['BFED'].fillna(method='ffill')
df = df.set_index(['Date']) # set date as index
df

,sp500,BK,BLK,BPOP,BX,C,CARV,CME,EVR,FIG,FII,GBL,LAZ,NDAQ,NTRS,PFS,STT,BFED
Date,,,,,,,,,,,,,,,,,,
2008-11-21,-0.007003,-0.000727,0.005141,0.004530,-0.000589,0.006967,0.006623,0.011028,0.023485,0.000000,0.004233,-0.004569,0.025546,0.003688,0.004829,-0.001145,0.006058,-0.001238
2008-11-24,0.001250,0.013061,0.004382,0.015291,0.009218,0.010086,0.000000,0.012436,0.016359,-0.002525,0.025379,-0.000794,0.001367,-0.006957,0.005442,0.011521,0.005198,0.000013
2008-11-25,-0.000829,0.002981,0.014912,-0.007634,0.010856,0.003820,0.000000,0.009372,0.014835,-0.003783,0.012602,0.019399,0.003737,0.006573,-0.001035,0.000577,0.001535,0.000013
2008-11-26,0.003328,0.004725,0.004249,0.015152,-0.000904,0.013585,0.000000,0.015810,0.045612,0.005083,0.014336,0.001838,0.009919,0.010204,0.008923,0.003462,0.013059,-0.001214
2008-11-28,0.006223,0.013078,-0.000730,0.003100,-0.002417,0.015885,0.000000,0.011341,0.000405,0.008974,0.010943,0.023514,0.009780,0.013264,0.009699,0.004643,0.012291,0.054545
2008-12-01,-0.005974,-0.009262,-0.004738,-0.010154,-0.013514,-0.008364,0.000000,0.000236,-0.017764,0.010363,-0.012472,-0.008212,-0.016490,-0.012932,-0.010520,-0.003482,0.002902,-0.001189
2008-12-02,-0.003480,-0.004229,-0.000501,-0.008204,0.000597,-0.000556,0.000000,0.006182,-0.010774,-0.005148,-0.011010,-0.013284,0.023034,0.002008,-0.000443,-0.024280,-0.003506,-0.005441
2008-12-03,0.002646,0.003995,-0.001536,0.002749,0.003896,-0.005877,0.000000,0.001077,-0.019661,0.011658,-0.000315,0.010403,-0.000593,0.005834,-0.005004,-0.005070,-0.002983,0.000000
2008-12-04,-0.001507,-0.006196,0.001037,0.002751,0.002402,-0.001287,0.027397,0.006264,0.009743,-0.015287,0.001263,0.017730,-0.005697,-0.001120,-0.002206,-0.001130,-0.002068,0.083871


<p>We also calculate the ten day cumulative results per Firm: </p>

In [12]:
sums={col:((1+df[col][0])*(1+df[col][1])*(1+df[col][2])*(1+df[col][3])*(1+df[col][4])*(1+df[col][5])*(1+df[col][6])*(1+df[col][7])*(1+df[col][8])*(1+df[col][9])*(1+df[col][10]))-1 for col in df}
sums_df=pd.DataFrame(sums,index=['Cumulative'])
df=df.append(sums_df)
df

,BFED,BK,BLK,BPOP,BX,C,CARV,CME,EVR,FIG,FII,GBL,LAZ,NDAQ,NTRS,PFS,STT,sp500
Date,,,,,,,,,,,,,,,,,,
2008-11-21,-0.001238,-0.000727,0.005141,0.004530,-0.000589,0.006967,0.006623,0.011028,0.023485,0.000000,0.004233,-0.004569,0.025546,0.003688,0.004829,-0.001145,0.006058,-0.007003
2008-11-24,0.000013,0.013061,0.004382,0.015291,0.009218,0.010086,0.000000,0.012436,0.016359,-0.002525,0.025379,-0.000794,0.001367,-0.006957,0.005442,0.011521,0.005198,0.001250
2008-11-25,0.000013,0.002981,0.014912,-0.007634,0.010856,0.003820,0.000000,0.009372,0.014835,-0.003783,0.012602,0.019399,0.003737,0.006573,-0.001035,0.000577,0.001535,-0.000829
2008-11-26,-0.001214,0.004725,0.004249,0.015152,-0.000904,0.013585,0.000000,0.015810,0.045612,0.005083,0.014336,0.001838,0.009919,0.010204,0.008923,0.003462,0.013059,0.003328
2008-11-28,0.054545,0.013078,-0.000730,0.003100,-0.002417,0.015885,0.000000,0.011341,0.000405,0.008974,0.010943,0.023514,0.009780,0.013264,0.009699,0.004643,0.012291,0.006223
2008-12-01,-0.001189,-0.009262,-0.004738,-0.010154,-0.013514,-0.008364,0.000000,0.000236,-0.017764,0.010363,-0.012472,-0.008212,-0.016490,-0.012932,-0.010520,-0.003482,0.002902,-0.005974
2008-12-02,-0.005441,-0.004229,-0.000501,-0.008204,0.000597,-0.000556,0.000000,0.006182,-0.010774,-0.005148,-0.011010,-0.013284,0.023034,0.002008,-0.000443,-0.024280,-0.003506,-0.003480
2008-12-03,0.000000,0.003995,-0.001536,0.002749,0.003896,-0.005877,0.000000,0.001077,-0.019661,0.011658,-0.000315,0.010403,-0.000593,0.005834,-0.005004,-0.005070,-0.002983,0.002646
2008-12-04,0.083871,-0.006196,0.001037,0.002751,0.002402,-0.001287,0.027397,0.006264,0.009743,-0.015287,0.001263,0.017730,-0.005697,-0.001120,-0.002206,-0.001130,-0.002068,-0.001507


In [13]:
#Some Dataframe Manipulations:
sp500=df['sp500'] #keep the values of SP500
#create new columns
df['ttest']=""
df['pvalue']=""

#Create t-statistic and pvalue per column
sample=df.as_matrix([df.columns[0:17]]) #sample of companies
sp500=df.as_matrix([df.columns[17]])    #value of sp500
for i in range(len(sample)):
    #select row (event day) 
    sam=sample[[i], :] #select row in sample
    sam=sam.transpose()
    t=float(sp500[[i], :]) #select row in sp500
    Event_day=stats.ttest_1samp(sam,t)
    ttest= np.array(Event_day[0], dtype=float)
    pvalue = np.array(Event_day[1], dtype=float)
    ttest=ttest[0]
    pvalue=pvalue[0]
    ttest
    pvalue
    df['ttest'][i]=float(ttest)
    df['pvalue'][i]=float(pvalue)

df

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,BFED,BK,BLK,BPOP,BX,C,CARV,CME,EVR,FIG,FII,GBL,LAZ,NDAQ,NTRS,PFS,STT,sp500,ttest,pvalue
Date,,,,,,,,,,,,,,,,,,,,
2008-11-21,-0.001238,-0.000727,0.005141,0.004530,-0.000589,0.006967,0.006623,0.011028,0.023485,0.000000,0.004233,-0.004569,0.025546,0.003688,0.004829,-0.001145,0.006058,-0.007003,6.3352,9.90536e-06
2008-11-24,0.000013,0.013061,0.004382,0.015291,0.009218,0.010086,0.000000,0.012436,0.016359,-0.002525,0.025379,-0.000794,0.001367,-0.006957,0.005442,0.011521,0.005198,0.001250,2.89505,0.0105487
2008-11-25,0.000013,0.002981,0.014912,-0.007634,0.010856,0.003820,0.000000,0.009372,0.014835,-0.003783,0.012602,0.019399,0.003737,0.006573,-0.001035,0.000577,0.001535,-0.000829,3.36992,0.00389996
2008-11-26,-0.001214,0.004725,0.004249,0.015152,-0.000904,0.013585,0.000000,0.015810,0.045612,0.005083,0.014336,0.001838,0.009919,0.010204,0.008923,0.003462,0.013059,0.003328,2.38265,0.0299344
2008-11-28,0.054545,0.013078,-0.000730,0.003100,-0.002417,0.015885,0.000000,0.011341,0.000405,0.008974,0.010943,0.023514,0.009780,0.013264,0.009699,0.004643,0.012291,0.006223,1.52714,0.146251
2008-12-01,-0.001189,-0.009262,-0.004738,-0.010154,-0.013514,-0.008364,0.000000,0.000236,-0.017764,0.010363,-0.012472,-0.008212,-0.016490,-0.012932,-0.010520,-0.003482,0.002902,-0.005974,-0.455121,0.655137
2008-12-02,-0.005441,-0.004229,-0.000501,-0.008204,0.000597,-0.000556,0.000000,0.006182,-0.010774,-0.005148,-0.011010,-0.013284,0.023034,0.002008,-0.000443,-0.024280,-0.003506,-0.003480,0.0888125,0.930333
2008-12-03,0.000000,0.003995,-0.001536,0.002749,0.003896,-0.005877,0.000000,0.001077,-0.019661,0.011658,-0.000315,0.010403,-0.000593,0.005834,-0.005004,-0.005070,-0.002983,0.002646,-1.59031,0.131328
2008-12-04,0.083871,-0.006196,0.001037,0.002751,0.002402,-0.001287,0.027397,0.006264,0.009743,-0.015287,0.001263,0.017730,-0.005697,-0.001120,-0.002206,-0.001130,-0.002068,-0.001507,1.57627,0.134527


<h2 id="Question-2">Question 2<a class="anchor-link" href="#Question-2">&#182;</a></h2><i><p>The authors define the cumulative abnormal returns (CAR) of firm $i$ as:</p>
$$ \mathrm{CAR}[0, n]_i = \sum_{t=0}^{n}\mathrm{AR}_{it}$$<p>where $\mathrm{CAR}[0, n]_i$ is the cumulative abnormal return for firm $i$ for event days 0 through $n$.</p>
<p>The term $\mathrm{AR}_{it}$ is calculated as follows:</p>
$$\mathrm{AR}_{it} = R_{it} - [\hat{\alpha_i} + \hat{\beta_i}R_{mt}]$$<p>where $\mathrm{AR}_{it}$ is the abnormal return for firm $i$ on event day $t$, $R_{it}$ is the actual return on firm $i$ for event day $t$, and $R_{mt}$ is the return on the market for event day $t$, with the market return represented by the return on the S&amp;P 500 index. Therefore to find $\mathrm{AR}_{it}$ you need to estimate $\hat{\alpha_i}$ and $\hat{\beta_i}$. You can do that by performing a regression:</p>
$$R_{it} = \alpha_i + \beta_iR_{mt} +\epsilon_{it}$$<p>on a pre-event period of 250 trading days ending 30 days prior to event day 0.</p>
<p>After having obtained the $\mathrm{CAR}[0, n]_i$, you will estimate the following equation:</p>
$$\mathrm{CAR}_i = \alpha + \beta x_i + \epsilon_i$$<p>where $\mathrm{CAR}_i$ is either $\mathrm{CAR}[0, 0]$, $\mathrm{CAR}[0, 1]$, or $\mathrm{CAR}[0, 10]$ for
firm $i$, and $x_i$ is a measure of connections for firm $i$.</p> <p>Your answer will include the estimate for $\beta$ when $x_i$ counts the schedule connections or when it counts the personal connections, when the dependent variable is $\mathrm{CAR}[0, 0]$, $\mathrm{CAR}[0, 1]$, or $\mathrm{CAR}[0, 10]$. Note: that means that you will run separate regressions for the schedule and the personal connections, in addition to the regressions for estimating $\hat{\alpha_i}$ and $\hat{\beta_i}$. Your anwer will be similar in spirit to Table 3 of the original paper.</p>
</i>

In [14]:
returns2=returns[returns['ticker'].isin(abbr)] #take the returns from acemoglus repliacations files
returns2=returns2.T    #transpose the dataset
returns2=returns2.drop(['index']) #drop row
header = returns2.iloc[0] # Create a new variable called 'header' from the first row of the dataset
returns2 = returns2[1:] # Replace the dataframe with a new one which does not contain the first row
returns2=returns2.rename(columns = header)# Rename the dataframe's column values with the header variable
#calculate Percent changes
returns3=returns2.pct_change()
#keep 250 days
df1=returns3.loc['ri26oct07':'ri9oct08']
df1['day']=range(1, len(df1) + 1 ,1) #column with day number
df1.head()

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,BK,BFED,BLK,BX,CARV,C,CME,EVR,FII,FIG,GBL,LAZ,NDAQ,NTRS,BPOP,PFS,STT,day
ri26oct07,0.008140,0.002754,0.016806,0.000411,0.000000,0.033957,-0.013612,0.057600,0.011117,0.020951,0.036827,0.054503,0.014907,0.018772,0.028075,0.045554,0.038158,1
ri29oct07,-0.006418,-0.002747,-0.008268,-0.003562,0.000000,0.001409,-0.001505,0.001966,0.009268,0.059440,-0.005221,0.037303,0.042644,0.005649,0.000000,-0.026037,-0.007351,2
ri30oct07,-0.000208,-0.005509,0.004472,-0.008250,-0.016803,-0.013588,-0.008899,0.005985,0.012815,-0.016831,-0.018331,-0.048509,-0.002824,-0.007623,0.004707,-0.022991,0.003575,3
ri31oct07,0.018132,-0.003728,0.046950,0.005546,0.038053,-0.004987,0.015453,0.028967,0.026750,-0.009375,0.032685,0.019494,0.013444,0.013612,-0.011246,0.008300,0.014885,4
ri1nov07,-0.038076,-0.009195,-0.031408,-0.021922,-0.020195,-0.068019,-0.025936,-0.015829,-0.038138,-0.110273,-0.055879,-0.043821,-0.023769,-0.049461,-0.029383,-0.057440,-0.075467,5


<p>Download also data for SP500 and merge them with our data: </p>

In [15]:
#take 250 days for SP500
s500 = pdr.get_data_yahoo(symbols='GSPC', start=datetime(2007, 10, 7),end=datetime(2008, 10, 9))
s500=pd.DataFrame(s500)
s500=s500[['Close','Open']]
s500['sp500']=(s500['Close']-s500['Open'])/s500['Open'] #return
del s500['Close']
del s500['Open']
s500['day']=range(1, len(df1) + 1 ,1) #column with day number
s500
#merge
df1=pd.merge(left=df1,right=s500, how='left', left_on=df1['day'], right_on=s500['day'])
df1

,BK,BFED,BLK,BX,CARV,C,CME,EVR,FII,FIG,GBL,LAZ,NDAQ,NTRS,BPOP,PFS,STT,day_x,sp500,day_y
0,0.008140,0.002754,0.016806,0.000411,0.000000,0.033957,-0.013612,0.057600,0.011117,0.020951,0.036827,0.054503,0.014907,0.018772,0.028075,0.045554,0.038158,1,0.004098,1
1,-0.006418,-0.002747,-0.008268,-0.003562,0.000000,0.001409,-0.001505,0.001966,0.009268,0.059440,-0.005221,0.037303,0.042644,0.005649,0.000000,-0.026037,-0.007351,2,0.008230,2
2,-0.000208,-0.005509,0.004472,-0.008250,-0.016803,-0.013588,-0.008899,0.005985,0.012815,-0.016831,-0.018331,-0.048509,-0.002824,-0.007623,0.004707,-0.022991,0.003575,3,0.002865,3
3,0.018132,-0.003728,0.046950,0.005546,0.038053,-0.004987,0.015453,0.028967,0.026750,-0.009375,0.032685,0.019494,0.013444,0.013612,-0.011246,0.008300,0.014885,4,0.007658,4
4,-0.038076,-0.009195,-0.031408,-0.021922,-0.020195,-0.068019,-0.025936,-0.015829,-0.038138,-0.110273,-0.055879,-0.043821,-0.023769,-0.049461,-0.029383,-0.057440,-0.075467,5,-0.009600,5
5,0.004469,-0.013920,-0.005734,-0.031435,-0.025181,-0.020255,0.001206,-0.050082,-0.009181,0.016957,0.009238,-0.008529,0.026783,0.018884,-0.068361,-0.022841,0.009085,6,-0.004040,6
6,0.004661,-0.017947,-0.019067,-0.006258,0.025832,-0.048501,0.004613,0.040353,-0.028315,-0.090799,-0.021359,-0.011173,0.020060,0.010983,-0.017818,0.007562,0.021499,7,-0.004040,7
7,0.018136,-0.001894,0.006647,-0.011277,0.017385,-0.022842,0.025052,0.024656,0.027620,-0.007669,0.008998,-0.004250,0.022210,0.041825,0.001066,0.033337,0.043541,8,-0.004040,8
8,-0.051989,-0.005359,-0.023120,-0.021478,-0.035497,-0.047606,-0.013600,-0.012460,-0.035850,-0.020665,-0.060420,-0.052920,-0.032790,-0.028676,-0.017057,-0.048109,-0.039203,9,-0.004040,9
9,0.005026,-0.008980,-0.007021,0.040569,0.000000,-0.015263,0.003481,0.002023,-0.020385,0.020415,-0.003491,0.042941,-0.038319,0.022006,-0.061822,0.049153,0.008791,10,-0.009697,10


In [16]:
#remember abbreviations and names of companies
comp=companies[['ticker','name']]
comp

,ticker,name
57,BFED,BEACON FED.BANC.INCORP.
63,BK,BANK OF NY.MELLON CORP.
70,BLK,BLACKROCK INCO.
83,BPOP,POPULAR INCO.
89,BX,THE BLACKSTONE GROUP LP.
92,C,CITIGROUP INCO.
100,CARV,CARVER BANCORP INCO.
140,CME,CME GROUP INCO.
232,EVR,EVERCORE PARTNERS INCO.
275,FIG,FORTRESS INV.GP.LLC.


<p> Create and Dataframe with the coefficients a,b per Firm: </p>

In [17]:
Param=pd.DataFrame(index=['Intercept','sp500'], columns=abbr)
#Rit=αi+βiRmt+ϵit
BFED = smf.ols("BFED ~ sp500", data=df1).fit()
Param['BFED']=BFED.params

BK = smf.ols("BK ~ sp500", data=df1).fit()
Param['BK']=BK.params


BLK = smf.ols("BLK ~ sp500", data=df1).fit()
Param['BLK']=BLK.params

BPOP = smf.ols("BPOP ~ sp500", data=df1).fit()
Param['BPOP']=BPOP.params

BX = smf.ols("BX ~ sp500", data=df1).fit()
Param['BX']=BX.params

C = smf.ols("C ~ sp500", data=df1).fit()
Param['C']=C.params

CARV = smf.ols("CARV ~ sp500", data=df1).fit()
Param['CARV']=CARV.params

CME = smf.ols("CME ~ sp500", data=df1).fit()
Param['CME']=CME.params

EVR = smf.ols("EVR ~ sp500", data=df1).fit()
Param['EVR']=EVR.params

FIG = smf.ols("FIG ~ sp500", data=df1).fit()
Param['FIG']=FIG.params

FII = smf.ols("FII ~ sp500", data=df1).fit()
Param['FII']=FII.params

GBL = smf.ols("GBL ~ sp500", data=df1).fit()
Param['GBL']=GBL.params

LAZ = smf.ols("LAZ ~ sp500", data=df1).fit()
Param['LAZ']=LAZ.params

NDAQ = smf.ols("NDAQ ~ sp500", data=df1).fit()
Param['NDAQ']=NDAQ.params

NTRS = smf.ols("NTRS ~ sp500", data=df1).fit()
Param['NTRS']=NTRS.params

PFS = smf.ols("PFS ~ sp500", data=df1).fit()
Param['PFS']=PFS.params

STT = smf.ols("STT ~ sp500", data=df1).fit()
Param['STT']=STT.params

Param


ticker,BFED,BK,BLK,BPOP,BX,C,CARV,CME,EVR,FIG,FII,GBL,LAZ,NDAQ,NTRS,PFS,STT
Intercept,-0.000839,-0.001709,-0.000219,-0.001131,-0.003397,-0.003807,-0.002965,-0.001997,-0.002090,-0.005461,-0.001527,-0.001076,-0.000789,-0.002085,-0.000778,-0.000531,-0.001066
sp500,0.015661,0.072419,0.036574,-0.131890,-0.045483,-0.175425,0.095417,-0.124478,-0.152754,-0.167933,0.074159,-0.194237,-0.158331,-0.214039,0.013097,0.009977,0.150085


In [18]:
###############################################################################
##create Dataframe to put values ARit=Rit−[αi^+βi^Rmt]
###############################################################################
df2=returns2.loc['ri20nov08':'ri8dec08'] #keep days of interest
df2=df2.drop('ri27nov08') #acemoglu's replication file has also this value exactly the same with ri26nov08, so we delete it
df2=df2.pct_change() #calculate returns
df2=df2.drop('ri20nov08') #delete the row that it was used to construct growth rates
df2['day']=range(1, len(df2) + 1 ,1) #column with day number to help merge with sp500 values
#take 250 days for SP500
s500 = pdr.get_data_yahoo(symbols='GSPC', start=datetime(2008, 11, 21),end=datetime(2008, 12, 8))
s500=pd.DataFrame(s500)
s500=s500[['Close','Open']]
s500['sp500']=(s500['Close']-s500['Open'])/s500['Open'] #return
del s500['Close']
del s500['Open']
s500['day']=range(1, len(df2) + 1 ,1) #column with day number
#merge
df2=pd.merge(left=df2,right=s500, how='left', left_on=df2['day'], right_on=s500['day'])
del df2['day_y'], df2['day_x']

AR=pd.DataFrame(index=range(11), columns=abbr)
for i in abbr:
    AR[i]= df2[i] -(Param[i][0]+Param[i][1]*df2['sp500'])
AR


ticker,BFED,BK,BLK,BPOP,BX,C,CARV,CME,EVR,FIG,FII,GBL,LAZ,NDAQ,NTRS,PFS,STT
0,0.008559,0.058262,0.090154,0.075449,-0.082067,-0.185595,-0.005643,0.132036,0.026647,-0.088675,0.041068,0.188085,0.119572,0.083029,0.100047,0.025278,0.100062
1,-0.010516,0.242752,0.063071,0.066000,0.390035,0.599083,-0.007825,0.155895,0.098524,0.192131,0.069291,0.083438,0.150439,0.143346,0.156336,0.128336,0.122137
2,-0.000437,-0.081711,0.008436,0.016830,0.008634,0.039945,-0.072916,0.008311,0.123853,0.014371,-0.035132,-0.012770,0.023356,0.032800,-0.053376,-0.021523,0.034833
3,0.029633,0.003419,0.059682,0.020722,0.031209,0.176301,-0.062594,0.070309,0.241535,0.275657,0.032014,0.037824,0.158290,0.145452,0.068387,0.013358,0.066803
4,0.058394,0.011791,0.008952,0.014225,-0.002170,0.175474,0.005255,-0.000092,-0.032577,0.011917,0.047474,-0.002178,0.038685,-0.005835,0.034419,0.002801,0.040584
5,-0.033284,-0.140803,-0.141024,-0.144565,-0.109628,-0.222533,0.005351,-0.186591,-0.006670,-0.178927,-0.101398,-0.171067,-0.189417,-0.128852,-0.140320,-0.158656,-0.153813
6,-0.036323,0.055841,0.061204,0.096037,-0.038630,0.112938,0.008538,-0.005507,0.081584,-0.004347,0.101538,0.012495,-0.010375,0.054532,0.083269,0.080448,0.052169
7,-0.040190,0.091222,0.074283,0.038048,0.134436,0.088755,0.001965,0.059458,0.003692,-0.244792,0.053640,0.118231,0.033537,0.025757,0.069388,0.029056,-0.009444
8,0.085079,-0.032105,-0.048692,-0.042763,-0.048921,-0.053951,0.005167,-0.027371,0.041788,0.012355,-0.048462,-0.029526,-0.044099,0.060227,0.014692,-0.034967,0.024202
9,0.001127,0.058399,0.109504,-0.004695,0.096994,0.042480,0.004717,0.096113,0.060289,-0.066115,0.102180,0.089768,0.014993,0.072260,0.096536,0.056293,0.083135


<p> Now we will Calculate the several CARs: </p>

In [19]:
###############################################################################
##create Dataframe to put values for CAR[0,n]i=∑t=0nARit
###############################################################################
CAR_0=AR.ix[0,]
CAR_0to1=AR.ix[0,]+AR.ix[1,]
CAR_0to10=AR.ix[0,]+AR.ix[1,]+AR.ix[2,]+AR.ix[3,]+AR.ix[4,]+AR.ix[5,]+AR.ix[6,]+AR.ix[7,]+AR.ix[8,]+AR.ix[9,]+AR.ix[10,]
CAR_0=CAR_0.to_frame()
CAR_0=CAR_0.rename(columns = {0:'CAR_0'})
CAR_0to1=CAR_0to1.to_frame()
CAR_0to1=CAR_0to1.rename(columns = {0:'CAR_0to1'})
CAR_0to10=CAR_0to10.to_frame()
CAR_0to10=CAR_0to10.rename(columns = {0:'CAR_0to10'})
CARs=pd.concat([CAR_0,CAR_0to1,CAR_0to10],axis=1)
CARs['ticker']=CARs.index #column to help merging
CARs

,CAR_0,CAR_0to1,CAR_0to10,ticker
ticker,,,,
BFED,0.008559,-0.001957,0.062185,BFED
BK,0.058262,0.301015,0.286471,BK
BLK,0.090154,0.153225,0.360695,BLK
BPOP,0.075449,0.141449,0.159380,BPOP
BX,-0.082067,0.307968,0.484065,BX
C,-0.185595,0.413487,0.883079,C
CARV,-0.005643,-0.013468,-0.119265,CARV
CME,0.132036,0.287932,0.463615,CME
EVR,0.026647,0.125171,0.722512,EVR


<p>According to TABLE 3 of the paper, <b>control variables</b> include squares and cubics in size (log of total assets), profitability (return on equity), and leverage (total debt #to total capital as of 2008. So we access those variables via  stata replication file Financial Data.dta </p>

In [20]:
financial=companies[companies.columns[[0,1,2,3,4]]]
financial

,ticker,name,ta2008,roe2008,tdtc2008
57,BFED,BEACON FED.BANC.INCORP.,1.011947e+06,-2.790000,73.870003
63,BK,BANK OF NY.MELLON CORP.,2.375120e+08,5.090000,45.820000
70,BLK,BLACKROCK INCO.,1.991301e+07,6.640000,8.360000
83,BPOP,POPULAR INCO.,3.852526e+07,-49.759998,67.989998
89,BX,THE BLACKSTONE GROUP LP.,8.412488e+06,-30.070000,6.160000
92,C,CITIGROUP INCO.,1.938470e+09,-28.469999,82.540001
100,CARV,CARVER BANCORP INCO.,7.883080e+05,7.510000,44.410000
140,CME,CME GROUP INCO.,4.815866e+07,4.620000,16.420000
232,EVR,EVERCORE PARTNERS INCO.,6.778740e+05,-2.490000,29.150000
275,FIG,FORTRESS INV.GP.LLC.,1.169669e+06,-236.380005,82.589996


In [21]:
financial['ta2008_log']=np.log(financial['ta2008']) #log variable according table3 of the paper
financial['size']=financial['ta2008_log']
financial['size_sq']=financial['size']**2 # square
financial['size_cb']=financial['size']**3 #cubic
financial['profitability']=financial['roe2008'] 
financial['profitability_sq']=financial['profitability']**2 
financial['profitability_cb']=financial['profitability']**3
financial['leverage']=financial['tdtc2008'] 
financial['leverage_sq']=financial['leverage']**2
financial['leverage_cb']=financial['leverage']**3
del financial['ta2008'], financial['ta2008_log'], financial['roe2008'], financial['tdtc2008']
financial

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,ticker,name,size,size_sq,size_cb,profitability,profitability_sq,profitability_cb,leverage,leverage_sq,leverage_cb
57,BFED,BEACON FED.BANC.INCORP.,13.827387,191.196624,2643.749673,-2.790000,7.784100,-2.171764e+01,73.870003,5456.777344,403092.156250
63,BK,BANK OF NY.MELLON CORP.,19.285729,371.939332,7173.121047,5.090000,25.908102,1.318722e+02,45.820000,2099.472412,96197.820312
70,BLK,BLACKROCK INCO.,16.806884,282.471345,4747.463078,6.640000,44.089600,2.927549e+02,8.360000,69.889595,584.276978
83,BPOP,POPULAR INCO.,17.466825,305.089965,5328.952927,-49.759998,2476.057373,-1.232086e+05,67.989998,4622.639648,314293.281250
89,BX,THE BLACKSTONE GROUP LP.,15.945228,254.250290,4054.078806,-30.070000,904.204895,-2.718944e+04,6.160000,37.945599,233.744873
92,C,CITIGROUP INCO.,21.385165,457.325275,9779.976395,-28.469999,810.540833,-2.307610e+04,82.540001,6812.851562,562332.812500
100,CARV,CARVER BANCORP INCO.,13.577644,184.352421,2503.071569,7.510000,56.400105,4.235648e+02,44.410000,1972.248047,87587.539062
140,CME,CME GROUP INCO.,17.690012,312.936508,5535.850438,4.620000,21.344398,9.861112e+01,16.420000,269.616394,4427.101562
232,EVR,EVERCORE PARTNERS INCO.,13.426717,180.276722,2420.524470,-2.490000,6.200100,-1.543825e+01,29.150000,849.722473,24769.410156
275,FIG,FORTRESS INV.GP.LLC.,13.972231,195.223249,2727.704405,-236.380005,55875.507812,-1.320785e+07,82.589996,6821.107422,563355.250000


In [22]:
#merge CARs and financial variables via tickers
financial=pd.merge(CARs,financial,how='left', on=['ticker'])
financial

,CAR_0,CAR_0to1,CAR_0to10,ticker,name,size,size_sq,size_cb,profitability,profitability_sq,profitability_cb,leverage,leverage_sq,leverage_cb
0,0.008559,-0.001957,0.062185,BFED,BEACON FED.BANC.INCORP.,13.827387,191.196624,2643.749673,-2.790000,7.784100,-2.171764e+01,73.870003,5456.777344,403092.156250
1,0.058262,0.301015,0.286471,BK,BANK OF NY.MELLON CORP.,19.285729,371.939332,7173.121047,5.090000,25.908102,1.318722e+02,45.820000,2099.472412,96197.820312
2,0.090154,0.153225,0.360695,BLK,BLACKROCK INCO.,16.806884,282.471345,4747.463078,6.640000,44.089600,2.927549e+02,8.360000,69.889595,584.276978
3,0.075449,0.141449,0.159380,BPOP,POPULAR INCO.,17.466825,305.089965,5328.952927,-49.759998,2476.057373,-1.232086e+05,67.989998,4622.639648,314293.281250
4,-0.082067,0.307968,0.484065,BX,THE BLACKSTONE GROUP LP.,15.945228,254.250290,4054.078806,-30.070000,904.204895,-2.718944e+04,6.160000,37.945599,233.744873
5,-0.185595,0.413487,0.883079,C,CITIGROUP INCO.,21.385165,457.325275,9779.976395,-28.469999,810.540833,-2.307610e+04,82.540001,6812.851562,562332.812500
6,-0.005643,-0.013468,-0.119265,CARV,CARVER BANCORP INCO.,13.577644,184.352421,2503.071569,7.510000,56.400105,4.235648e+02,44.410000,1972.248047,87587.539062
7,0.132036,0.287932,0.463615,CME,CME GROUP INCO.,17.690012,312.936508,5535.850438,4.620000,21.344398,9.861112e+01,16.420000,269.616394,4427.101562
8,0.026647,0.125171,0.722512,EVR,EVERCORE PARTNERS INCO.,13.426717,180.276722,2420.524470,-2.490000,6.200100,-1.543825e+01,29.150000,849.722473,24769.410156
9,-0.088675,0.103456,0.037814,FIG,FORTRESS INV.GP.LLC.,13.972231,195.223249,2727.704405,-236.380005,55875.507812,-1.320785e+07,82.589996,6821.107422,563355.250000


In [23]:
#create Schedule and Personal connections and merge with the Financial Data
Schedule_Connections={'name':pd.Series(['BANK OF NY.MELLON CORP.','BEACON FED.BANC.INCORP.',
                                        'BLACKROCK INCO.','CME GROUP INCO.','EVERCORE PARTNERS INCO.',
                                        'FEDERATED INVRS.INCO.','LAZARD LTD.','NORTHERN TRUST CORP.',
                                        'PROVIDENT FINL.SVS.INCO.','STATE STREET CORP.',
                                        'THE BLACKSTONE GROUP LP.','THE NASDAQ OMX GP.INCO.']),
                     'Sced_conn':pd.Series([7,1,13,2,1,1,1,1,2,1,6,2])}

Personal_Connections={'name':pd.Series(['BLACKROCK INCO.','CARVER BANCORP INCO.','CITIGROUP INCO.',
                                        'FORTRESS INV.GP.LLC.','GAMCO INVESTORS INCO.','POPULAR INCO.',
                                        'THE BLACKSTONE GROUP LP.','THE NASDAQ OMX GP.INCO.']),
                      'Person_conn':pd.Series([2,1,2,1,1,1,4,1])}

Sced_conn = pd.DataFrame(Schedule_Connections)
Person_conn = pd.DataFrame(Personal_Connections)

In [29]:
Sced_conn #Schedule_Connections

,Sced_conn,name
0,7,BANK OF NY.MELLON CORP.
1,1,BEACON FED.BANC.INCORP.
2,13,BLACKROCK INCO.
3,2,CME GROUP INCO.
4,1,EVERCORE PARTNERS INCO.
5,1,FEDERATED INVRS.INCO.
6,1,LAZARD LTD.
7,1,NORTHERN TRUST CORP.
8,2,PROVIDENT FINL.SVS.INCO.
9,1,STATE STREET CORP.


In [25]:
Person_conn #Personal_Connections

,Person_conn,name
0,2,BLACKROCK INCO.
1,1,CARVER BANCORP INCO.
2,2,CITIGROUP INCO.
3,1,FORTRESS INV.GP.LLC.
4,1,GAMCO INVESTORS INCO.
5,1,POPULAR INCO.
6,4,THE BLACKSTONE GROUP LP.
7,1,THE NASDAQ OMX GP.INCO.


In [26]:
#merge with financial data and fill with zeros
financial=pd.merge(financial,Person_conn,how='left', on=['name'])
financial=pd.merge(financial,Sced_conn,how='left', on=['name'])
financial['Sced_conn']=financial['Sced_conn'].fillna(0) # fill missing values with nulls
financial['Person_conn']=financial['Person_conn'].fillna(0) # fill missing values with nulls
financial

,CAR_0,CAR_0to1,CAR_0to10,ticker,name,size,size_sq,size_cb,profitability,profitability_sq,profitability_cb,leverage,leverage_sq,leverage_cb,Person_conn,Sced_conn
0,0.008559,-0.001957,0.062185,BFED,BEACON FED.BANC.INCORP.,13.827387,191.196624,2643.749673,-2.790000,7.784100,-2.171764e+01,73.870003,5456.777344,403092.156250,0.0,1.0
1,0.058262,0.301015,0.286471,BK,BANK OF NY.MELLON CORP.,19.285729,371.939332,7173.121047,5.090000,25.908102,1.318722e+02,45.820000,2099.472412,96197.820312,0.0,7.0
2,0.090154,0.153225,0.360695,BLK,BLACKROCK INCO.,16.806884,282.471345,4747.463078,6.640000,44.089600,2.927549e+02,8.360000,69.889595,584.276978,2.0,13.0
3,0.075449,0.141449,0.159380,BPOP,POPULAR INCO.,17.466825,305.089965,5328.952927,-49.759998,2476.057373,-1.232086e+05,67.989998,4622.639648,314293.281250,1.0,0.0
4,-0.082067,0.307968,0.484065,BX,THE BLACKSTONE GROUP LP.,15.945228,254.250290,4054.078806,-30.070000,904.204895,-2.718944e+04,6.160000,37.945599,233.744873,4.0,6.0
5,-0.185595,0.413487,0.883079,C,CITIGROUP INCO.,21.385165,457.325275,9779.976395,-28.469999,810.540833,-2.307610e+04,82.540001,6812.851562,562332.812500,2.0,0.0
6,-0.005643,-0.013468,-0.119265,CARV,CARVER BANCORP INCO.,13.577644,184.352421,2503.071569,7.510000,56.400105,4.235648e+02,44.410000,1972.248047,87587.539062,1.0,0.0
7,0.132036,0.287932,0.463615,CME,CME GROUP INCO.,17.690012,312.936508,5535.850438,4.620000,21.344398,9.861112e+01,16.420000,269.616394,4427.101562,0.0,2.0
8,0.026647,0.125171,0.722512,EVR,EVERCORE PARTNERS INCO.,13.426717,180.276722,2420.524470,-2.490000,6.200100,-1.543825e+01,29.150000,849.722473,24769.410156,0.0,1.0
9,-0.088675,0.103456,0.037814,FIG,FORTRESS INV.GP.LLC.,13.972231,195.223249,2727.704405,-236.380005,55875.507812,-1.320785e+07,82.589996,6821.107422,563355.250000,1.0,0.0


In [27]:
#Create Dataframe to be filled, we create also columns to fill with pvalues
regressions=('CAR_0_sced','CAR_0_sced_pvalues','CAR_0_pers','CAR_0_pers_pvalues', 'CAR_0to1_sced','CAR_0to1_sced_pvalues', 'CAR_0to1_pers', 'CAR_0to1_pers_pvalues', 'CAR_0to10_sced','CAR_0to10_sced_pvalues', 'CAR_0to10_pers', 'CAR_0to10_pers_pvalues')
Final=pd.DataFrame(index=['Intercept','Sced_conn', 'Person_conn','size','size_sq', 'size_cb','profitability', 'profitability_sq', 'profitability_cb', 'leverage', 'leverage_sq', 'leverage_cb', 'Number of firms', 'R-squared'], columns=regressions)
Final

,CAR_0_sced,CAR_0_sced_pvalues,CAR_0_pers,CAR_0_pers_pvalues,CAR_0to1_sced,CAR_0to1_sced_pvalues,CAR_0to1_pers,CAR_0to1_pers_pvalues,CAR_0to10_sced,CAR_0to10_sced_pvalues,CAR_0to10_pers,CAR_0to10_pers_pvalues
Intercept,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sced_conn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Person_conn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
size,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
size_sq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
size_cb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
profitability,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
profitability_sq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
profitability_cb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
leverage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#Run the regressions and fill the above Dataframe in spirit to Table 3 of the original paper.
CAR_0_sced = smf.ols('CAR_0 ~ Sced_conn+size+size_sq+size_cb+profitability+profitability_sq+profitability_cb+leverage+leverage_sq+leverage_cb', data=financial).fit()
Final['CAR_0_sced']=CAR_0_sced.params #take the coefficients and fill the dataframe
Final.ix[-2,'CAR_0_sced']=int(CAR_0_sced.nobs) #fill Number of Observations/Firms
Final.ix[-1,'CAR_0_sced']=CAR_0_sced.rsquared #fill Last Row with Rsquared Values
Final['CAR_0_sced_pvalues']=CAR_0_sced.pvalues # Fill Column with P-Values

CAR_0_pers = smf.ols('CAR_0 ~ Person_conn+size+size_sq+size_cb+profitability+profitability_sq+profitability_cb+leverage+leverage_sq+leverage_cb', data=financial).fit()
Final['CAR_0_pers']=CAR_0_pers.params
Final.ix[-2,'CAR_0_pers']=int(CAR_0_pers.nobs)
Final.ix[-1,'CAR_0_pers']=CAR_0_pers.rsquared
Final['CAR_0_pers_pvalues']=CAR_0_pers.pvalues

CAR_0to1_sced = smf.ols('CAR_0to1 ~ Sced_conn+size+size_sq+size_cb+profitability+profitability_sq+profitability_cb+leverage+leverage_sq+leverage_cb', data=financial).fit()
Final['CAR_0to1_sced']=CAR_0to1_sced.params
Final.ix[-2,'CAR_0to1_sced']=int(CAR_0to1_sced.nobs)
Final.ix[-1,'CAR_0to1_sced']=CAR_0to1_sced.rsquared
Final['CAR_0to1_sced_pvalues']=CAR_0to1_sced.pvalues

CAR_0to1_pers = smf.ols('CAR_0to1 ~ Person_conn+size+size_sq+size_cb+profitability+profitability_sq+profitability_cb+leverage+leverage_sq+leverage_cb', data=financial).fit()
Final['CAR_0to1_pers']=CAR_0to1_pers.params
Final.ix[-2,'CAR_0to1_pers']=int(CAR_0to1_pers.nobs)
Final.ix[-1,'CAR_0to1_pers']=CAR_0to1_pers.rsquared
Final['CAR_0to1_pers_pvalues']=CAR_0to1_pers.pvalues

CAR_0to10_sced = smf.ols('CAR_0to10 ~ Sced_conn+size+size_sq+size_cb+profitability+profitability_sq+profitability_cb+leverage+leverage_sq+leverage_cb', data=financial).fit()
Final['CAR_0to10_sced']=CAR_0to10_sced.params
Final.ix[-2,'CAR_0to10_sced']=int(CAR_0to10_sced.nobs)
Final.ix[-1,'CAR_0to10_sced']=CAR_0to10_sced.rsquared
Final['CAR_0to10_sced_pvalues']=CAR_0to10_sced.pvalues

CAR_0to10_pers = smf.ols('CAR_0to10 ~ Person_conn+size+size_sq+size_cb+profitability+profitability_sq+profitability_cb+leverage+leverage_sq+leverage_cb', data=financial).fit()
Final['CAR_0to10_pers']=CAR_0to10_pers.params
Final.ix[-2,'CAR_0to10_pers']=int(CAR_0to10_pers.nobs)
Final.ix[-1,'CAR_0to10_pers']=CAR_0to10_pers.rsquared
Final['CAR_0to10_pers_pvalues']=CAR_0to10_pers.pvalues

Final

,CAR_0_sced,CAR_0_sced_pvalues,CAR_0_pers,CAR_0_pers_pvalues,CAR_0to1_sced,CAR_0to1_sced_pvalues,CAR_0to1_pers,CAR_0to1_pers_pvalues,CAR_0to10_sced,CAR_0to10_sced_pvalues,CAR_0to10_pers,CAR_0to10_pers_pvalues
Intercept,1.851962e+01,0.061850,2.042942e+01,0.068917,-6.249949e+00,0.667390,1.481961e+00,0.927119,-9.559712e+00,0.779514,-2.718029e+00,0.945975
Sced_conn,2.627054e-03,0.740523,NaN,NaN,-8.221707e-03,0.549389,NaN,NaN,-1.783135e-02,0.581138,NaN,NaN
Person_conn,NaN,NaN,1.529287e-02,0.651589,NaN,NaN,4.664154e-02,0.422326,NaN,NaN,3.272216e-02,0.815408
size,-3.557628e+00,0.049459,-3.924878e+00,0.058966,1.031570e+00,0.692018,-4.661661e-01,0.874978,1.606546e+00,0.792913,2.751451e-01,0.970052
size_sq,2.225442e-01,0.040682,2.453583e-01,0.051129,-5.545444e-02,0.718166,3.637354e-02,0.837533,-8.844967e-02,0.806627,-7.494383e-03,0.986362
size_cb,-4.579377e-03,0.033512,-5.044278e-03,0.044535,1.012966e-03,0.734960,-8.435679e-04,0.809680,1.658765e-03,0.813736,3.033558e-05,0.997202
profitability,1.849388e-04,0.853290,5.329471e-04,0.622611,-1.739189e-04,0.919002,2.104167e-04,0.907123,2.013815e-04,0.960179,-3.209116e-05,0.994263
profitability_sq,-9.804650e-06,0.671350,-1.366645e-05,0.546205,-1.594720e-05,0.686633,-1.521120e-05,0.687084,-2.338071e-05,0.801167,-1.356529e-05,0.883927
profitability_cb,-2.603938e-08,0.793208,-4.712759e-08,0.624961,-5.858629e-08,0.731025,-5.580590e-08,0.729448,-6.886602e-08,0.863466,-1.705622e-08,0.965769
leverage,2.591177e-02,0.087962,2.795985e-02,0.095310,1.504499e-02,0.515723,3.547441e-02,0.186462,4.591698e-02,0.406138,7.078885e-02,0.274697
